<a href="https://colab.research.google.com/github/csabiu/KAML-2025/blob/main/KAML_catalogues_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install library for accessing numerous astronomical datasets

In [ ]:
!pip install astroquery

In [ ]:
from astroquery.sdss import SDSS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# an SQL query for SDSS data
query = """
SELECT TOP 10000
       p.ra, p.dec,
       p.u, p.g, p.r, p.i, p.z,
       s.z as redshift,
       s.class as specClass
FROM SpecObjAll s
JOIN PhotoObjAll p ON s.bestObjID = p.objid
WHERE s.class in ('STAR', 'GALAXY', 'QSO')
  AND p.clean = 1
  AND p.u between 0 AND 30  -- Exclude obviously invalid magnitudes
  AND p.g between 0 AND 30
  AND p.r between 0 AND 30
  AND p.i between 0 AND 30
  AND p.z between 0 AND 30
ORDER BY NEWID()  -- Randomize the sample
"""

In [ ]:
res = SDSS.query_sql(query) # sometimes this produces an error message - try re-running it OR skip to the next cells

In [ ]:
df_sdss = res.to_pandas()
df_sdss.head()

If there are errors accessing the SDSS data then uncomment the following lines....

In [ ]:
#df_sdss = pd.read_csv("https://archive.kasi.re.kr/bigdata/temp/KAML_catalogues.csv", header=0)
#df_sdss.head()

In [ ]:
# Plot histogram of the 5 SDSS magnitudes
df_sdss[["u", "g", "r", "i", "z"]].hist(bins=20)
plt.show()

In [ ]:
# Plot the distribution of classes
df_sdss["specClass"].value_counts().plot(kind="pie")
plt.show()

In [ ]:
# lets make 4 additional pieces of information: color
df_sdss['u_g'] = df_sdss['u'] - df_sdss['g'] # u-g
df_sdss['g_r'] = df_sdss['g'] - df_sdss['r'] # g-r
df_sdss['r_i'] = df_sdss['r'] - df_sdss['i'] # r-i
df_sdss['i_z'] = df_sdss['i'] - df_sdss['z'] # i-z

# View the new color-index columns
df_sdss[['u_g', 'g_r', 'r_i', 'i_z']].head()

In [ ]:
# Plot a color-magnitude diagram
class_colors = {
    'STAR': 'blue',
    'GALAXY': 'red',
    'QSO': 'green'
}

plt.figure(figsize=(6, 5))

for cls, color in class_colors.items():
    subset = df_sdss[df_sdss['specClass'] == cls]
    plt.scatter(subset['g_r'], subset['g'], c=color, label=cls, alpha=0.2, s=1.5)

plt.gca().invert_yaxis()
plt.xlabel('g - r (Color Index)')
plt.ylabel('g (Magnitude)')
plt.title('Color-Magnitude Diagram by Class')
plt.xlim([-1,6])
plt.legend()
plt.show()

The above color-magnitude diagram is not enough to clearly seperate the 3 types of object. So lets try and do some machine learning on this data to help us predict the correct classification

In [ ]:
from sklearn.model_selection import train_test_split

features = ["u", "g", "r", "i", "z", "u_g", "g_r", "r_i", "i_z"]  # magnitudes and colors
labels = "specClass"

X = df_sdss[features].values
y = df_sdss[labels].values

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,   # use 20% of the data for testing
                                                    random_state=42,)
                                                    #stratify=y)

print("Training inputs: ",X_train.shape)
print("Training outputs: ",y_train.shape)
print("Testing inputs: ",X_test.shape)
print("Testing outputs: ",y_test.shape)

**Task: Classification**

We want to train a machine to classify the phoometric sources as either Star, Galaxy or QSO

**Decision Tree**

A decision tree is a simple binary classification that aims to maximise the entropy between the seperated objects after each 'decision'

Decisions are binary and hierachical, they can be many levels deep.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier


dt = DecisionTreeClassifier(max_depth=2,random_state=42,)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)

In [ ]:
#from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus

#dot_data=export_graphviz(dt)
dot_data = export_graphviz(
    dt,
    out_file=None,
    feature_names=features,
    class_names=["STAR", "GALAXY", "QSO"],
    filled=True,
    rounded=True)

graph = pydotplus.graph_from_dot_data(dot_data)

Image(graph.create_png())

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

Below are brief definitions of **precision**, **recall**, and **F1**. They provide simple metrics with which to assess the performance of classification methods. We can use these to test hyper-parameters of an alogorithm or compare alogorithms with each other.

---

## Precision
**Precision** is the fraction of predicted positives (i.e., the objects your model labeled as positive) that are actually positive:

$$
\text{Precision} = \frac{TP}{TP + FP}
$$



- **TP** = True Positives: Items correctly labeled as positive.  
- **FP** = False Positives: Items incorrectly labeled as positive.

---

## Recall
**Recall** (or sensitivity) is the fraction of actual positives that are correctly identified:

$$
\text{Recall} = \frac{TP}{TP + FN}
$$

- **TP** = True Positives.  
- **FN** = False Negatives: Items that were positive but incorrectly labeled as negative.

---

## F1 Score
The **F1 score** is the harmonic mean of precision and recall:

$$
F1 = 2 \times \frac{\text{precision} \times \text{recall}}{\text{precision} + \text{recall}}
$$

F1 ranges between 0 and 1, with 1 indicating perfect precision and recall.

In [ ]:
# This is how you can compute the above metrics...
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test, y_pred, average='macro')
print('precision:',precision)
recall    = recall_score(y_test, y_pred, average='macro')
print('recall:',recall)
f1    = f1_score(y_test, y_pred, average='macro')
print('f1:',f1)

# Assignment 1:
* Create a plot of the **precision** as a function of **max depth** of the decision tree. try depths values from 1-20. show precision for test and train samples
* Do the same for **recall**

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score


... blank ...



A Random Forest is a machine learning algorithm that belongs to the family of ensemble methods.  It operates by constructing multiple decision trees during training and outputting the class that is the mode of the classes (classification) or mean/average prediction (regression) of the individual trees.  Essentially, it leverages the "wisdom of the crowd" – many relatively weak learners (decision trees) combine to form a powerful predictor.

Here's a breakdown of key aspects:

**1. Decision Trees:**  The foundation of a Random Forest is the decision tree.  Each tree is built on a random subset of the training data (bootstrap aggregating or bagging).  This means that each tree doesn't see the entire dataset, introducing diversity.  Furthermore, at each node of a tree, a random subset of features is considered for the split. This further decorrelates the trees, as they will be based on different features and different subsets of the training data.

**2. Bagging (Bootstrap Aggregating):**  This technique involves creating multiple subsets of the original training data by randomly sampling with replacement.  This means some data points might appear multiple times in a subset, while others might be left out entirely.  Each subset is used to train a different decision tree.  This randomness helps to reduce the variance of the model.

**3. Random Subspace (Feature Randomness):** At each node of each decision tree, the algorithm considers only a random subset of the features when deciding on the best split.  This prevents individual trees from becoming overly dependent on a single feature, increasing the diversity of the forest.

**4. Prediction:** To make a prediction for a new data point, the Random Forest feeds the data point through each decision tree in the forest. Each tree produces a prediction, and the final prediction is an aggregate of the individual tree predictions.  For classification, this is typically the most frequent class (mode); for regression, it's the average of the predictions.


**Advantages of Random Forests:**

* **High accuracy:** Random Forests often achieve high accuracy compared to single decision trees due to the combination of multiple trees and the reduced overfitting.
* **Robustness to outliers:** The use of multiple trees and bootstrapping makes the model less sensitive to outliers in the data.
* **Handles high dimensionality well:** Feature randomness helps the model handle a large number of features efficiently.
* **Can handle missing values:** Random Forests can be adapted to deal with missing data without requiring imputation.
* **Provides feature importance:**  The algorithm can estimate the relative importance of each feature in the prediction, allowing for feature selection and better understanding of the model.

**Disadvantages:**

* **Computational cost:** Training many decision trees can be computationally expensive, especially with large datasets.
* **Black box nature:** While feature importance is available, it can be difficult to interpret the complete decision-making process of a Random Forest.


In the provided code example, a Random Forest isn't directly used. However, the analysis explores decision trees and their performance, illustrating some of the fundamental concepts that apply to Random Forests as well, particularly regarding feature selection, model evaluation (precision, recall, F1-score), and the impact of model parameters (max depth).  You can easily replace the `DecisionTreeClassifier` with `RandomForestClassifier` to apply these same evaluation methods to a Random Forest.



In [ ]:
# Choose a range of tree counts to test
n_estimators_range = range(1, 15, 1)

precisions = []
recalls = []
f1_scores = []
precisions_train = []
recalls_train = []
f1_scores_train = []

# Use 'binary' or 'macro' depending on your task
average_type = 'macro'  # or 'weighted' or 'binary'

for n in n_estimators_range:
    clf = RandomForestClassifier(
        n_estimators=n,
        max_depth=8,
        random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_pred_train = clf.predict(X_train)

    precisions.append(precision_score(y_test, y_pred, average=average_type))
    recalls.append(recall_score(y_test, y_pred, average=average_type))
    f1_scores.append(f1_score(y_test, y_pred, average=average_type))

    precisions_train.append(precision_score(y_train, y_pred_train, average=average_type))
    recalls_train.append(recall_score(y_train, y_pred_train, average=average_type))
    f1_scores_train.append(f1_score(y_train, y_pred_train, average=average_type))

# Plotting
plt.figure(figsize=(8, 5))
plt.plot(n_estimators_range, precisions, label='Precision',color='b')
plt.plot(n_estimators_range, recalls, label='Recall',color='r')
plt.plot(n_estimators_range, f1_scores, label='F1 Score',color='g')

plt.plot(n_estimators_range, precisions_train,ls='--',color='b')
plt.plot(n_estimators_range, recalls_train,ls='--',color='r')
plt.plot(n_estimators_range, f1_scores_train,ls='--',color='g')

plt.xlabel('Number of Trees (n_estimators)')
plt.ylabel('Score')
plt.title(f'Random Forest Performance vs. Number of Trees ({average_type} average)')
plt.legend()
plt.grid(True)
plt.show()


A neural network is made up of layers:

**Input Layer**

Each node represents a feature of your data (e.g., color indices like g−rg−r, r−ir−i, etc.).

**Hidden Layers**

One or more layers that process and transform the input. Each neuron in a hidden layer performs:

$$z=w_1x_1+w_2x_2+...+w_nx_n+b$$

followed by a nonlinear activation function like **sigmoid** or **tanh** etc etc

**Output Layer**

Produces the final prediction. For example, a softmax output layer can give probabilities for classes like STAR, GALAXY, or QSO.

The softmax function is used in the output layer of a neural network for multi-class classification. It converts a vector of raw scores (called logits) into probabilities that sum to 1 — making it easy to interpret the output as the model's confidence in each class.

If you have output scores $$z_1​,z_2​,...,z_K$$​ for K classes, the softmax function turns them into probabilities $$p_1,p_2,...,p_K$$​ like this:
$$\text{softmax}(z_i)=\frac{e^{z_i}}{\sum_i^K e^{z_j}}$$


In [ ]:
# prompt: A simple neural network for star, galaxy, QSO classification

import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

# Encode the labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Convert labels to categorical
y_categorical = keras.utils.to_categorical(y_encoded, num_classes=3)  # 3 classes


# Define the model
model = keras.Sequential([
    keras.layers.Dense(128, activation='sigmoid', input_shape=(X.shape[1],)),  # Input layer
    keras.layers.Dense(64, activation='sigmoid'),  # Hidden layer
    keras.layers.Dense(3, activation='softmax')  # Output layer (3 classes)
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# Split the data: train 80% - test 20%
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42, stratify=y_encoded)


# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)



In [ ]:
# Plot the training history
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Make predictions
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_test_decoded = np.argmax(y_test, axis=1)


# Convert predictions back to original labels
y_pred_labels = le.inverse_transform(y_pred)
y_test_labels = le.inverse_transform(y_test_decoded)


print(classification_report(y_test_labels,y_pred_labels))

# Assignment 2:



*   Try different choices of number of layers and neurons per layer
*   What is the best **precision** and **recall** values you can obtain?



